In [ ]:
#Preprocessing del Dataset di Flickr

with open('image_features.pkl', 'rb') as f:
        image_features = pickle.load(f)

 def text_preprocessing(data):
        data['caption'] = data['caption'].apply(lambda x: x.lower())
        data['caption'] = data['caption'].apply(lambda x: " ".join([word for word in x.split() if len(word) > 1]))
        data['caption'] = "startseq " + data['caption'] + " endseq"
        return data

df_captions = text_preprocessing(df_captions)
captions = df_captions['caption'].tolist()

import numpy as np
images, filename_index = [], []
for i, filename in enumerate(df_captions.image):
        if filename in image_features.keys():
            images.append(image_features[filename])
            filename_index.append(i)

filenames = df_captions["image"].iloc[filename_index].values
captions = df_captions["caption"].iloc[filename_index].values
images = np.array(images)


from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(captions)

sequences = tokenizer.texts_to_sequences(captions)
maxlen = len(max(sequences, key=len))

vocab_size = len(tokenizer.word_index) + 1



val_size = int(len(sequences) * 0.2)
test_size = int(len(sequences) * 0.2)

def split_test_val_train(sequences, val_size, test_size):
    return (sequences[:test_size], sequences[test_size:test_size+val_size], sequences[test_size+val_size:])

txt_test, txt_val, txt_train = split_test_val_train(sequences, val_size, test_size)
img_test, img_val, img_train = split_test_val_train(images, val_size, test_size)
filename_test, filename_val, filename_train = split_test_val_train(filenames, val_size, test_size)


prova = sequences[0]



for i in range(1, len(prova)):
    seq_in = prova[:i]
    seq_out = prova[i]
    print(seq_in, seq_out)

 def prepare_data(sequences, images):
        print("# Captions = {}".format(len(sequences)))

        X_seq, X_img, y_seq = [], [], []
        for seq, img in zip(sequences, images):
           for i in range(1, len(seq)):

                seq_in, seq_out = seq[:i], seq[i]
                seq_in = pad_sequences([seq_in], maxlen=maxlen).flatten()
                seq_out = to_categorical(seq_out, num_classes=vocab_size)
                X_seq.append(seq_in)
                X_img.append(img)
                y_seq.append(seq_out)

           X_seq = np.array(X_seq)
           X_img = np.array(X_img)
           y_seq = np.array(y_seq)

        print("\nShapes:\n{} {} {}".format(X_seq.shape, X_img.shape, y_seq.shape))
        return (X_seq, X_img, y_seq)